In [43]:
# Auto update impots
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Daily Back Fill

In [44]:
from op_analytics.datasources.github.dataaccess import Github
prs_df = Github.PRS.read_polars()
comments_df = Github.PR_COMMENTS.read_polars()
reviews_df = Github.PR_REVIEWS.read_polars()

2025-02-05 14:31:17 [info     ] Reading data from 'github/github_prs_v1' with filters min_date=None, max_date=None, date_range_spec=None filename=dailydata.py lineno=78 process=36713
2025-02-05 14:31:22 [info     ] registered view: 'github_github_prs_v1' using uri wildcard 'gs://oplabs-tools-data-sink/github/github_prs_v1/dt=*/out.parquet' filename=client.py lineno=291 process=36713
┌──────────────────────────────┐
│             name             │
│           varchar            │
├──────────────────────────────┤
│ github_github_pr_comments_v1 │
│ github_github_pr_reviews_v2  │
│ github_github_prs_v1         │
└──────────────────────────────┘

2025-02-05 14:31:28 [info     ] Reading data from 'github/github_pr_comments_v1' with filters min_date=None, max_date=None, date_range_spec=None filename=dailydata.py lineno=78 process=36713
2025-02-05 14:31:34 [info     ] registered view: 'github_github_pr_comments_v1' using uri wildcard 'gs://oplabs-tools-data-sink/github/github_pr_comments_v1/d

In [45]:
# print the schemas and the names of the dataframes
print("Pull Requests Schema:")
print(prs_df.schema)
print("\nPR Comments Schema:")
print(comments_df.schema) 
print("\nPR Reviews Schema:")
print(reviews_df.schema)

Pull Requests Schema:
Schema({'html_url': String, 'number': Int64, 'state': String, 'labels': List(Struct({'id': Int64, 'name': String, 'description': String})), 'draft': Boolean, 'created_at': String, 'updated_at': String, 'closed_at': String, 'merged_at': String, 'author_association': String, 'user': Struct({'login': String, 'id': Int64}), 'title': String, 'body': String, 'base': Struct({'label': String, 'ref': String}), 'head': Struct({'label': String, 'ref': String}), 'assignee': Struct({'login': String, 'id': Int64}), 'assignees': List(Struct({'login': String, 'id': Int64})), 'requested_reviewers': List(Struct({'login': String, 'id': Int64})), 'requested_teams': List(Struct({'id': Int64, 'name': String, 'parent': Struct({'id': Int64, 'name': String})})), 'repo': String, 'dt': Date})

PR Comments Schema:
Schema({'pr_number': Int64, 'id': Int64, 'body': String, 'author_association': String, 'created_at': String, 'updated_at': String, 'user': Struct({'login': String, 'id': Int64}), '

In [46]:
# Assert all max dates of the 3 datasets are the same
assert prs_df["dt"].max() == comments_df["dt"].max() == reviews_df["dt"].max(), "Max dates should be equal across all datasets"

In [47]:
from op_analytics.datasources.github.metrics.compute import compute_all_metrics
detailed_metrics = compute_all_metrics(
    prs_df=prs_df,
    comments_df=comments_df,
    reviews_df=reviews_df,
)
detailed_metrics

repo,period_start,period_end,new_prs,merged_prs,closed_prs,active_prs,median_time_to_first_review_hours,median_time_to_first_non_bot_comment_hours,median_time_to_merge_hours,total_comments,total_reviews,unique_commenters,unique_reviewers,unique_contributors,approved_prs,rejected_prs,review_requested_prs,period_type,approval_ratio,merge_ratio,closed_ratio,comment_intensity,review_intensity,active_ratio,response_time_ratio,contributor_engagement
str,datetime[μs],datetime[μs],u32,u32,u32,u32,f64,f64,f64,u32,u32,u32,u32,u32,u32,u32,u32,str,f64,f64,f64,f64,f64,f64,f64,f64
"""optimism""",2020-02-11 00:00:00,2020-02-18 00:00:00,8,8,8,null,0.0,null,0.0,null,3,null,3,3,3,0,0,"""rolling_week""",0.375,1.0,1.0,null,0.375,null,0.0,0.375
"""optimism""",2020-02-12 00:00:00,2020-02-19 00:00:00,7,7,7,8,0.0,0.0,0.0,null,3,null,3,3,3,0,0,"""rolling_week""",0.428571,1.0,1.0,null,0.428571,1.142857,0.0,0.428571
"""optimism""",2020-02-13 00:00:00,2020-02-20 00:00:00,5,4,4,7,0.0,0.0,5.0,null,13,null,4,3,5,0,0,"""rolling_week""",1.0,0.8,0.8,null,2.6,1.4,0.0,0.6
"""optimism""",2020-02-14 00:00:00,2020-02-21 00:00:00,11,9,9,10,0.5,2.5,5.0,null,20,null,4,3,8,0,0,"""rolling_week""",0.727273,0.818182,0.818182,null,1.818182,0.909091,0.5,0.272727
"""optimism""",2020-02-15 00:00:00,2020-02-22 00:00:00,16,11,11,7,2.5,2.5,10.5,null,24,null,4,4,10,0,0,"""rolling_week""",0.625,0.6875,0.6875,null,1.5,0.4375,0.238095,0.25
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""specs""",2024-02-02 00:00:00,2025-02-01 00:00:00,405,330,383,406,10.0,6.0,23.0,2067,2165,74,78,84,315,53,303,"""rolling_year""",0.777778,0.814815,0.945679,5.103704,5.345679,1.002469,0.26087,0.207407
"""specs""",2024-02-03 00:00:00,2025-02-02 00:00:00,406,329,382,405,10.0,6.0,23.0,2049,2149,74,78,84,314,53,304,"""rolling_year""",0.773399,0.810345,0.940887,5.046798,5.293103,0.997537,0.26087,0.206897
"""specs""",2024-02-04 00:00:00,2025-02-03 00:00:00,406,329,382,405,10.0,6.0,23.0,2048,2147,73,77,84,313,53,304,"""rolling_year""",0.770936,0.810345,0.940887,5.044335,5.288177,0.997537,0.26087,0.206897


In [50]:
from op_analytics.datasources.github.metrics.execute import execute_pull_repo_metrics
# Full pipeline
repo_metrics = execute_pull_repo_metrics()

2025-02-05 14:38:57 [info     ] Reading data from 'github/github_prs_v1' with filters min_date=None, max_date=None, date_range_spec=None filename=dailydata.py lineno=78 process=36713
2025-02-05 14:39:02 [info     ] registered view: 'github_github_prs_v1' using uri wildcard 'gs://oplabs-tools-data-sink/github/github_prs_v1/dt=*/out.parquet' filename=client.py lineno=291 process=36713
┌──────────────────────────────┐
│             name             │
│           varchar            │
├──────────────────────────────┤
│ github_github_pr_comments_v1 │
│ github_github_pr_reviews_v2  │
│ github_github_prs_v1         │
└──────────────────────────────┘

2025-02-05 14:39:08 [info     ] Reading data from 'github/github_pr_comments_v1' with filters min_date=None, max_date=None, date_range_spec=None filename=dailydata.py lineno=78 process=36713
2025-02-05 14:39:13 [info     ] registered view: 'github_github_pr_comments_v1' using uri wildcard 'gs://oplabs-tools-data-sink/github/github_pr_comments_v1/d

## Visualizations

In [48]:
import polars as pl
import plotly.express as px

def plot_repo_metric_over_time(
    combined_df: pl.DataFrame,
    repo_name: str,
    metric: str,
    x_axis: str = "period_start",
    markers: bool = True
) -> None:
    """
    Plot a single metric for one repo, across all rolling timeframes (e.g. 'rolling_week',
    'rolling_month', etc.), in a single timeseries chart.

    Args:
        combined_df: Polars DataFrame containing rows for multiple period_type 
                     (e.g. 'rolling_week', 'rolling_month', etc.). 
                     Must have columns:
                       - 'repo' (String)
                       - 'period_start' (Datetime)
                       - 'period_type' (String; e.g. 'rolling_30d' or 'rolling_week')
                       - <metric> (the numeric column to plot)
        repo_name: The repo to filter on (e.g. 'optimism', 'supersim', etc.)
        metric: The numeric column to plot (e.g. 'new_prs', 'total_comments')
        x_axis: Column name to use for the time axis (default: 'period_start')
        markers: Whether to show markers on the line chart (default: True)
    """
    # 1) Filter for the selected repo
    df_pd = (
        combined_df
        .filter(pl.col("repo") == repo_name)
        .sort([x_axis, "period_type"])
        .select(["period_type", x_axis, metric])
        .to_pandas()
    )

    # 2) Plot a single figure with color='period_type'
    fig = px.line(
        df_pd,
        x=x_axis,
        y=metric,
        color="period_type",
        title=f"{metric} over time for '{repo_name}' by all timeframes",
        markers=markers
    )

    fig.update_layout(
        xaxis_title=x_axis,
        yaxis_title=metric,
        legend_title="Timeframe"
    )
    fig.show()

plot_repo_metric_over_time(
    combined_df=detailed_metrics,
    repo_name="optimism",
    metric="closed_prs",
    x_axis="period_end",
    markers=True
)

In [49]:
import os
current_path = os.getcwd()
print(current_path)

csv_path = os.path.join(current_path,"github_pr_metrics.csv")
detailed_metrics.to_pandas().to_csv(csv_path)

/Users/dioptx/Documents/DS-Projects/op-analytics/notebooks/adhoc/platform_github_metrics
